# Clean Data

In this notebook, I undertake the task of cleaning data obtained from educational portals such as [Education Data Explorer](https://educationdata.urban.org/data-explorer), [Civil Rights Data](https://civilrightsdata.ed.gov/), and the [U.S. Department of Education](https://www2.ed.gov/about/inits/ed/edfacts/data-files/index.html). The aim is to prepare the data for further analysis by addressing inconsistencies, errors, and missing values.

Given the diverse origins of the data and its storage in various tabular formats, I employ tailored cleaning procedures for each dataset. This involves tasks such as standardizing column names, correcting data types, handling outliers, and imputing missing values.

Additionally, I implement quality checks and validation steps to ensure the integrity and accuracy of the cleaned data. By conducting thorough data cleaning processes, I strive to enhance the reliability and usability of the datasets for building informative dashboards.

In [1]:
import json

import pandas as pd
import numpy as np

# Load fips
with open("./data/fips.json") as file:
    fips_dict = json.load(file)
    
# Fips dictionary
fips_dict = {int(k):v for k,v in fips_dict.items()}

# Race dictionary
race_dict = {
    1: "White",
    2: "Black",
    3: "Hispanic",
    4: "Asian",
    5: "American Indian or Alaska Native",
}

sex_dict = {
    1: "Male",
    2: "Female",
}


## Demographic information

Certain metrics in the Google Looker Studio dashboard necessitate data regarding the population in the diverse states of America and their demographics for the analyzed period. This data is sourced from [KFF](https://www.kff.org/)

In [2]:
# I need the FIPS number for each state
inv_fips_dict = {v: int(k) for k, v in fips_dict.items()}

# Read demographic data for each year
df_dem_2011 = pd.read_csv("data/raw/kff/raw_data2011.csv", skiprows=2)
df_dem_2011["year"] = 2011
df_dem_2013 = pd.read_csv("data/raw/kff/raw_data2013.csv", skiprows=2)
df_dem_2013["year"] = 2013
df_dem_2015 = pd.read_csv("data/raw/kff/raw_data2015.csv", skiprows=2)
df_dem_2015["year"] = 2015
df_dem_2017 = pd.read_csv("data/raw/kff/raw_data2017.csv", skiprows=2)
df_dem_2017["year"] = 2017
df_dem_2021 = pd.read_csv("data/raw/kff/raw_data2021.csv", skiprows=2)
df_dem_2021["year"] = 2020

# Concatenate demographic data for all years
df_dem = pd.concat([df_dem_2011, df_dem_2013, df_dem_2015, df_dem_2017, df_dem_2021])

# Assign FIPS codes to each state
df_dem["fips"] = df_dem['Location'].apply(lambda x: inv_fips_dict[x] if x in inv_fips_dict else np.nan)
df_dem.dropna(subset=['fips'], inplace=True)
df_dem.fillna(0, inplace=True)
df_dem["fips"] = df_dem["fips"].astype(int)

# Add country and state information
df_dem['country'] = "United States"
df_dem["state"] = df_dem['Location']
df_dem = df_dem.rename(columns={
    'American Indian/Alaska Native': 'American Indian or Alaska Native',
    'Total': 'All'
})

# Assign date values
df_dem["day"] = 1
df_dem["month"] = 1
df_dem['date'] = pd.to_datetime(df_dem[["year", "month", "day"]])

# Reshape the dataframe to have population values by race
df_dem = df_dem.melt(id_vars=["year", 'date', 'country', "fips", "state"],
                     value_vars=['White', 'Black', 'Hispanic', 'American Indian or Alaska Native', 'All'],
                     var_name='race_name', value_name='population')
df_dem["population"] = df_dem["population"].astype(int)

# Save the demographic data to a CSV file
df_dem.to_csv("data/kff/population.csv", index=False)

## Enrollment data

I extract the enrollment data, transform it, and finally, load it into a CSV file. For this dataset, I have different files for each year (five files for 2011, 2013, 2015, 2017, and 2020).

In [3]:
for year in [2011, 2013, 2015, 2017, 2020]:
    
    # Load the data from the Urban Institute's Education Data portal
    enroll_df = pd.read_csv(f"https://educationdata.urban.org/csv/ccd/schools_ccd_enrollment_{year}.csv")
    
    if not enroll_df.empty:
        
        # Remove unwanted data and handle missing values
        enroll_df.loc[enroll_df["enrollment"] < 0, "enrollment"] = np.nan
        enroll_df.loc[enroll_df["fips"] < 0, "fips"] = np.nan
        enroll_df.loc[enroll_df["grade"] < 1, "grade"] = np.nan
        enroll_df.loc[enroll_df["grade"] > 13, "grade"] = np.nan
        
        # Add state, race, and sex name based on dictionaries
        enroll_df["state"] = enroll_df['fips'].apply(lambda x: fips_dict[x] if x in fips_dict else np.nan)
        enroll_df["race_name"] = enroll_df['race'].apply(lambda x: race_dict[x] if x in race_dict else np.nan)
        enroll_df["sex_name"] = enroll_df['sex'].apply(lambda x: sex_dict[x] if x in sex_dict else np.nan)
        
        # Drop rows with missing values in specific columns
        enroll_df.dropna(subset=["grade", 'enrollment', 'fips', 'state', 'race_name', 'sex_name'], inplace=True)
        
        # Fix datatype
        enroll_df["grade"] = enroll_df["grade"].astype(int)
        enroll_df["enrollment"] = enroll_df["enrollment"].astype(int)
        
        # Add new columns for country, day, and month
        enroll_df['country'] = "United States"
        enroll_df["day"] = 1
        enroll_df["month"] = 1
        enroll_df['date'] = pd.to_datetime(enroll_df[["year","month","day"]])
        
        # Aggregate data to a state level from a school level
        enroll_df = enroll_df.groupby(["year", 'date', 'country', "fips", "state", "grade", "race", 'race_name', "sex", 'sex_name'], as_index=False).agg({"enrollment": "sum"})
        
        # Save the cleaned data to a CSV file for the year
        enroll_df.to_csv(f"data/raw/urban_institute/ccd_schools_states_enrollment_{year}.csv", index=False)
        del enroll_df  # Clean up the dataframe from memory after saving
 

In [4]:
enrollment_list = []
for year in [2011, 2013, 2015, 2017, 2020]:
    
    # Read the CSV file for the current year and append it to the list
    enrollment_list.append(pd.read_csv(f"data/raw/urban_institute/ccd_schools_states_enrollment_{year}.csv"))

# Concatenate all DataFrames in the list into a single DataFrame
df_enrollment_out = pd.concat(enrollment_list)

# Write the concatenated DataFrame to a CSV file
df_enrollment_out.to_csv("data/urban_institute/ccd_schools_states_enrollment.csv", index=False)

I calculate the Diversity Index similar to the US census, and also how many enrollments are per capita. Both calculations are saved in a separate file to the previous one because Google Looker needs the data in a specific way.

In [5]:
# Calculate the diversity index (Simpson Index)

df_enrollment_last_year = df_enrollment_out.groupby(["year", 'country', "state", "race", 'race_name'], as_index=False).agg(
    {"enrollment": "sum"})

# Filter for data from the last year in the dataset
df_enrollment_last_year = df_enrollment_last_year[df_enrollment_last_year["year"] > 2019]

# Group by year, country, and state and aggregate total enrollment to obtain the enrollment by state
df_enrollment_total = df_enrollment_last_year.groupby(["year", 'country', "state"], as_index=False).agg({"enrollment": "sum"})
df_enrollment_total = df_enrollment_total.rename(columns={"enrollment": "enrollment_total"})

# Merge the total enrollment data back into the DataFrame
df_enrollment_last_year = df_enrollment_last_year.merge(right=df_enrollment_total, on=["year", 'country', "state"])

# Calculate the proportion of enrollment for each race within each state
df_enrollment_last_year["p"] = df_enrollment_last_year["enrollment"] / df_enrollment_last_year["enrollment_total"]

# Apply the Simpson Index formula (Σ(p^2))
df_enrollment_last_year["p2"] = df_enrollment_last_year["p"] ** 2
df_enrollment_last_year = df_enrollment_last_year.groupby(["year", 'country', "state"], as_index=False).agg({"enrollment": "sum", "p2": "sum"})

# Calculate the diversity index using the formula 1 - Σ(p^2)
df_enrollment_last_year["diversity_index"] = 1 - df_enrollment_last_year["p2"]

In [6]:
# Read population data from the provided CSV file
df_dem = pd.read_csv("data/kff/population.csv")

# Group population data by year, country, and state and aggregate total population
df_dem = df_dem.groupby(["year", 'country', "state"], as_index=False).agg({"population": "sum"})

# Filter for data from the last year in the dataset
df_dem = df_dem[df_dem["year"] > 2019]

# Merge enrollment data with population data using year, country, and state as keys
df_enrollment_last_year = df_enrollment_last_year.merge(right=df_dem, on=["year", 'country', "state"])

# Calculate enrollment per capita by dividing enrollment by population
df_enrollment_last_year["enroll_per_capita"] = df_enrollment_last_year["enrollment"] / df_enrollment_last_year["population"]

# Write the resulting DataFrame to a new CSV file
df_enrollment_last_year.to_csv("data/urban_institute/crdc_schools_states_enrollment_state_diversity_index.csv", index=False)

# Assessment information

I extract the assessment data, transform it, and finally, load it into a CSV file. For this dataset, I have different files for each year (five files for 2011, 2013, 2015, 2017, and 2020).

In [7]:
for year in [2011, 2013, 2015, 2017, 2020]:
    
    assessment_df = pd.read_csv(f"https://educationdata.urban.org/csv/edfacts/schools_edfacts_assessments_{year}.csv")
    
    if not assessment_df.empty:
        
        # Remove unwanted data and handle missing values
        assessment_df.loc[assessment_df["fips"] < 0, "fips"] = np.nan
        
        assessment_df = assessment_df[assessment_df["grade_edfacts"] == 99]
        assessment_df = assessment_df[assessment_df["lep"] == 99]
        assessment_df = assessment_df[assessment_df["homeless"] == 99]
        assessment_df = assessment_df[assessment_df["migrant"] == 99]
        assessment_df = assessment_df[assessment_df["disability"] == 99]
        assessment_df = assessment_df[assessment_df["econ_disadvantaged"] == 99]
        assessment_df = assessment_df[assessment_df["foster_care"] == 99]
        assessment_df = assessment_df[assessment_df["military_connected"] == 99]
        assessment_df = assessment_df[assessment_df["sex"] == 99]
        
        assessment_df = assessment_df[assessment_df["read_test_num_valid"] > 0]
        assessment_df = assessment_df[assessment_df["read_test_pct_prof_low"] > 0]
        assessment_df = assessment_df[assessment_df["read_test_pct_prof_midpt"] > 0]
        assessment_df = assessment_df[assessment_df["read_test_pct_prof_high"] > 0]
        
        assessment_df = assessment_df[assessment_df["math_test_num_valid"] > 0]
        assessment_df = assessment_df[assessment_df["math_test_pct_prof_low"] > 0]
        assessment_df = assessment_df[assessment_df["math_test_pct_prof_midpt"] > 0]
        assessment_df = assessment_df[assessment_df["math_test_pct_prof_high"] > 0]
        
        assessment_df = assessment_df[assessment_df["race"] < 80]
        assessment_df = assessment_df[assessment_df["race"] > 0]
        
        # Add state and race based on dictionaries
        assessment_df["state"] = assessment_df['fips'].apply(lambda x: fips_dict[x] if x in fips_dict else np.nan)
        assessment_df["race_name"] = assessment_df['race'].apply(lambda x: race_dict[x] if x in race_dict else np.nan)
        
        # Drop rows with missing values in specific columns
        assessment_df.dropna(subset=["grade_edfacts", 'fips', 'state', 'race_name'], inplace=True)
        
        # Fix datatype
        assessment_df["grade"] = assessment_df["grade_edfacts"].astype(int)
        assessment_df["read_test_num_valid"] = assessment_df["read_test_num_valid"].astype(int)
        assessment_df["math_test_num_valid"] = assessment_df["math_test_num_valid"].astype(int)
        
        # Add new columns for country, day, and month
        assessment_df['country'] = "United States"
        assessment_df["day"] = 1
        assessment_df["month"] = 1
        assessment_df['date'] = pd.to_datetime(assessment_df[["year","month","day"]])
          
        # Aggregate data to a state level from a school level
        assessment_df = assessment_df.groupby(["year", 'date', 'country', "fips", "state", "grade", "race", 'race_name'], as_index=False).agg({
            "read_test_num_valid": "sum",
            'read_test_pct_prof_low': "mean",
            'read_test_pct_prof_midpt': "mean",
            'read_test_pct_prof_high': "mean",
            
            "math_test_num_valid": "sum",
            'math_test_pct_prof_low': "mean",
            'math_test_pct_prof_midpt': "mean",
            'math_test_pct_prof_high': "mean",
        })
        
        # Save the cleaned data to a CSV file for the year
        assessment_df.to_csv(f"data/raw/urban_institute/edfacts_schools_states_assessments_{year}.csv", index=False)
        del assessment_df  # Clean up the dataframe from memory after saving

In [8]:
assessment_list = []
for year in [2011, 2013, 2015, 2017, 2020]:
    
    # Read the CSV file for the current year and append it to the list
    assessment_list.append(pd.read_csv(f"data/raw/urban_institute/edfacts_schools_states_assessments_{year}.csv"))
    
# Concatenate all DataFrames in the list into a single DataFrame
df_assessment_out = pd.concat(assessment_list)

# Fix datatype
df_assessment_out["read_test_num_valid"] = df_assessment_out["read_test_num_valid"].astype(int)
df_assessment_out["read_test_pct_prof_low"] = df_assessment_out["read_test_pct_prof_low"].astype(int)
df_assessment_out["read_test_pct_prof_midpt"] = df_assessment_out["read_test_pct_prof_midpt"].astype(int)
df_assessment_out["read_test_pct_prof_high"] = df_assessment_out["read_test_pct_prof_high"].astype(int)
df_assessment_out["math_test_num_valid"] = df_assessment_out["math_test_num_valid"].astype(int)
df_assessment_out["math_test_pct_prof_low"] = df_assessment_out["math_test_pct_prof_low"].astype(int)
df_assessment_out["math_test_pct_prof_midpt"] = df_assessment_out["math_test_pct_prof_midpt"].astype(int)
df_assessment_out["math_test_pct_prof_high"] = df_assessment_out["math_test_pct_prof_high"].astype(int)

# Write the concatenated DataFrame to a CSV file
df_assessment_out.to_csv("data/urban_institute/edfacts_schools_states_assessments.csv", index=False)

Create separate CSV files for the assessment data without racial information and for the grade balance graph in the Google Looker Studio dashboard.

In [9]:
# Aggregate assessment data without considering race
df_assessment_no_race = df_assessment_out.groupby(["year", "date", 'country', "state"], as_index=False).agg({
    "read_test_num_valid": "sum",
    "read_test_pct_prof_midpt": "median",
    "math_test_num_valid": "sum",
    "math_test_pct_prof_midpt": "median"
})

# Write the aggregated assessment data to a CSV file
df_assessment_no_race.to_csv("data/urban_institute/edfacts_schools_states_assessments_no_race.csv")

# Data for the balanced between Math and Reading graph. 
# Rename columns and reshape the DataFrame to have separate columns for Math and Reading exam grade medians
df_assessment_out["Maths Exam Grade Median"] = df_assessment_out["math_test_pct_prof_midpt"]
df_assessment_out["Reading Exam Grade Median"] = df_assessment_out["read_test_pct_prof_midpt"]
df_assessment_medians = pd.melt(df_assessment_out, id_vars=["year", "date", 'country', "state", "race_name"],
                               value_vars=["Maths Exam Grade Median", "Reading Exam Grade Median"], var_name="Subject",
                               value_name='Exam Grade', col_level=None, ignore_index=True)
# Write the reshaped DataFrame to a CSV file
df_assessment_medians.to_csv("data/urban_institute/edfacts_schools_states_assessments_medians.csv", index=False)

## Discipline Dataset

I extract the discipline data, transform it, and finally, load it into a CSV file. For this dataset, I have different files for each year (four files for 2011, 2013, 2015, 2017).

In [10]:
for year in [2011, 2013, 2015, 2017]:
    
    discipline_df = pd.read_csv(f"https://educationdata.urban.org/csv/crdc/schools_crdc_discipline_k12_{year}.csv")
    
    if not discipline_df.empty:
        
        # Remove unwanted data and handle missing values
        discipline_df.loc[discipline_df["fips"] < 0, "fips"] = np.nan
        
        # Set negative student suspension counts to zero
        discipline_df.loc[discipline_df["students_susp_in_sch"] < 0, "students_susp_in_sch"] = 0
        discipline_df.loc[discipline_df["students_susp_out_sch_single"] < 0, "students_susp_out_sch_single"] = 0
        discipline_df.loc[discipline_df["students_susp_out_sch_multiple"] < 0, "students_susp_out_sch_multiple"] = 0
        discipline_df.loc[discipline_df["expulsions_no_ed_serv"] < 0, "expulsions_no_ed_serv"] = 0
        discipline_df.loc[discipline_df["expulsions_with_ed_serv"] < 0, "expulsions_with_ed_serv"] = 0
        discipline_df.loc[discipline_df["expulsions_zero_tolerance"] < 0, "expulsions_zero_tolerance"] = 0
        discipline_df.loc[discipline_df["students_corporal_punish"] < 0, "students_corporal_punish"] = 0
        discipline_df.loc[discipline_df["students_arrested"] < 0, "students_arrested"] = 0
        discipline_df.loc[discipline_df["students_referred_law_enforce"] < 0, "students_referred_law_enforce"] = 0
        discipline_df.loc[discipline_df["transfers_alt_sch_disc"] < 0, "transfers_alt_sch_disc"] = 0
        
        # Fill NaN values with zero for certain columns
        discipline_df["students_susp_in_sch"] = discipline_df["students_susp_in_sch"].fillna(0)
        discipline_df["students_susp_out_sch_single"] = discipline_df["students_susp_out_sch_single"].fillna(0)
        discipline_df["students_susp_out_sch_multiple"] = discipline_df["students_susp_out_sch_multiple"].fillna(0)
        discipline_df["expulsions_no_ed_serv"] = discipline_df["expulsions_no_ed_serv"].fillna(0)
        discipline_df["expulsions_with_ed_serv"] = discipline_df["expulsions_with_ed_serv"].fillna(0)
        discipline_df["expulsions_zero_tolerance"] = discipline_df["expulsions_zero_tolerance"].fillna(0)
        discipline_df["students_corporal_punish"] = discipline_df["students_corporal_punish"].fillna(0)
        discipline_df["students_arrested"] = discipline_df["students_arrested"].fillna(0)
        discipline_df["students_referred_law_enforce"] = discipline_df["students_referred_law_enforce"].fillna(0)
        discipline_df["transfers_alt_sch_disc"] = discipline_df["transfers_alt_sch_disc"].fillna(0)
        
        # Convert certain columns to integer type
        discipline_df["students_susp_in_sch"] = discipline_df["students_susp_in_sch"].astype(int)
        discipline_df["students_susp_out_sch_single"] = discipline_df["students_susp_out_sch_single"].astype(int)
        discipline_df["students_susp_out_sch_multiple"] = discipline_df["students_susp_out_sch_multiple"].astype(int)
        discipline_df["expulsions_no_ed_serv"] = discipline_df["expulsions_no_ed_serv"].astype(int)
        discipline_df["expulsions_with_ed_serv"] = discipline_df["expulsions_with_ed_serv"].astype(int)
        discipline_df["expulsions_zero_tolerance"] = discipline_df["expulsions_zero_tolerance"].astype(int)
        discipline_df["students_corporal_punish"] = discipline_df["students_corporal_punish"].astype(int)
        discipline_df["students_arrested"] = discipline_df["students_arrested"].astype(int)
        discipline_df["students_referred_law_enforce"] = discipline_df["students_referred_law_enforce"].astype(int)
        discipline_df["transfers_alt_sch_disc"] = discipline_df["transfers_alt_sch_disc"].astype(int)
        
        # Filter the data for students with no disability or LEP status 
        discipline_df = discipline_df[discipline_df["disability"] == 99]
        discipline_df = discipline_df[discipline_df["lep"] == 99]
        
        # Create new columns for aggregated disciplinary actions
        
        # Combine out-of-school suspension counts
        discipline_df["students_susp_out_sch"] = discipline_df["students_susp_out_sch_multiple"] + discipline_df["students_susp_out_sch_single"]
        # Combine different types of expulsions
        discipline_df["students_expulsions"] = discipline_df["expulsions_no_ed_serv"] + discipline_df["expulsions_with_ed_serv"] + discipline_df["expulsions_zero_tolerance"]
        # Combine students arrested and referred to law enforcement
        discipline_df["students_arrested"] = discipline_df["students_arrested"] + discipline_df["students_referred_law_enforce"]
        
        # Map FIPS codes to state names
        discipline_df["state"] = discipline_df['fips'].apply(lambda x: fips_dict[x] if x in fips_dict else np.nan)
        # Map race codes to race names
        discipline_df["race_name"] = discipline_df['race'].apply(lambda x: race_dict[x] if x in race_dict else np.nan)
        # Map sex codes to sex names
        discipline_df["sex_name"] = discipline_df['sex'].apply(lambda x: sex_dict[x] if x in sex_dict else np.nan)
        
        # Drop rows with missing values in specific columns
        discipline_df.dropna(subset=['fips', 'state', 'race_name', 'sex_name'], inplace=True)
        
        # Add columns for country, day, and month
        discipline_df['country'] = "United States"
        discipline_df["day"] = 1
        discipline_df["month"] = 1
        discipline_df['date'] = pd.to_datetime(discipline_df[["year","month","day"]])
        
        # Aggregate data by year, date, country, FIPS code, state, race, and sex
        discipline_df = discipline_df.groupby(["year", 'date', 'country', "fips", "state", "race", 'race_name', "sex", 'sex_name'], as_index=False).agg({
            "students_susp_in_sch": "sum",
            "students_susp_out_sch": "sum",
            "students_expulsions": "sum",
            "students_corporal_punish": "sum",
            "students_arrested": "sum",
            "transfers_alt_sch_disc": "sum",
        })
        
        # Write the cleaned and aggregated data to a CSV file
        discipline_df.to_csv(f"data/raw/urban_institute/crdc_schools_states_discipline_{year}.csv", index=False)
        del discipline_df  # Delete the DataFrame to free up memory

/var/folders/50/q05bl7gj7jxb0qq6rb67fp4m0000gn/T/ipykernel_9159/524524948.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  discipline_df = pd.read_csv(f"https://educationdata.urban.org/csv/crdc/schools_crdc_discipline_k12_{year}.csv")
/var/folders/50/q05bl7gj7jxb0qq6rb67fp4m0000gn/T/ipykernel_9159/524524948.py:3: DtypeWarning: Columns (0,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  discipline_df = pd.read_csv(f"https://educationdata.urban.org/csv/crdc/schools_crdc_discipline_k12_{year}.csv")
/var/folders/50/q05bl7gj7jxb0qq6rb67fp4m0000gn/T/ipykernel_9159/524524948.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  discipline_df = pd.read_csv(f"https://educationdata.urban.org/csv/crdc/schools_crdc_discipline_k12_{year}.csv")
/var/folders/50/q05bl7gj7jxb0qq6rb67fp4m0000gn/T/ipykernel_9159/524524948.py:3: DtypeWarning: Columns (0) have mix

In [11]:
discipline_list = []
for year in [2011, 2013, 2015, 2017, 2020]:
        
    if year < 2020:
        discipline_list.append(pd.read_csv(f"data/raw/urban_institute/crdc_schools_states_discipline_{year}.csv"))
    else:
        # Read the discipline data for the year 2017 and copy as 2020
        df_temp = pd.read_csv("data/raw/urban_institute/crdc_schools_states_discipline_2017.csv")
        df_temp["year"] = 2020
        df_temp["day"] = 1
        df_temp["month"] = 1
        df_temp['date'] = pd.to_datetime(df_temp[["year","month","day"]])
        df_temp['date'] = df_temp['date'].dt.strftime("%Y-%m-%d")
        df_temp.drop(columns=['day', 'month'], inplace=True)
        discipline_list.append(df_temp)
        
df_discipline_out = pd.concat(discipline_list)

In [12]:
# Load the enrollment dataset to calculate metrics by number of enrollment
df_enrollment = pd.read_csv("data/urban_institute/ccd_schools_states_enrollment.csv")

# Merge discipline data with enrollment data
df_discipline_out = df_discipline_out.merge(right=df_enrollment,
                                            on=["year", 'date', 'country', 'fips', "state", "race", 'race_name', "sex", 'sex_name'],
                                            how="left")

# Calculate total behavioral incidents
df_discipline_out["behavioral_incidents"] = (
        df_discipline_out["students_susp_in_sch"] + df_discipline_out["students_susp_out_sch"] +
        df_discipline_out["students_expulsions"] + df_discipline_out["students_corporal_punish"] + 
        df_discipline_out["students_arrested"] + df_discipline_out["transfers_alt_sch_disc"]
)

# Write the DataFrame to a CSV file
df_discipline_out.to_csv("data/urban_institute/crdc_schools_states_discipline.csv", index=False)

Create separate CSV files for the discipline data without racial or sex information for the Disciplinary Actions per Enrollment in the Google Looker Studio dashboard.

In [13]:
# Aggregate discipline data at a state level
df_discipline_out = df_discipline_out.groupby(["year", 'date', 'country', 'fips', "state", "race", 'race_name'],
                                              as_index=False).agg({
    "students_susp_in_sch": "sum",
    "students_susp_out_sch": "sum",
    "students_expulsions": "sum",
    "students_corporal_punish": "sum",
    "students_arrested": "sum",
    "transfers_alt_sch_disc": "sum",
    "behavioral_incidents": "sum",
    "enrollment": "sum"
})

# Calculate disciplinary actions per enrollment without sex information
df_discipline_out["behavioral_prop_enrollment"] = df_discipline_out["behavioral_incidents"] / df_discipline_out[
    "enrollment"]

# Aggregate the data at a state level
df_discipline_agg = df_discipline_out.groupby(["year", 'date', 'country', 'fips', "state"], as_index=False).agg({
    "behavioral_prop_enrollment": "sum",
})
df_discipline_agg = df_discipline_agg.rename(columns={"behavioral_prop_enrollment": "behavioral_prop_enrollment_total"})

df_discipline_out = df_discipline_out.merge(right=df_discipline_agg, on=["year", 'date', 'country', 'fips', "state"])

# Calculate the percentage of disciplinary actions per enrollment
df_discipline_out["behavioral_prop_enrollment"] = (df_discipline_out["behavioral_prop_enrollment"] * 100 / df_discipline_out["behavioral_prop_enrollment_total"])

# Convert to integer and drop the intermediate column
df_discipline_out["behavioral_prop_enrollment"] = df_discipline_out["behavioral_prop_enrollment"].astype(int)
df_discipline_out.drop(columns="behavioral_prop_enrollment_total", inplace=True)

# Write the DataFrame to a CSV file
df_discipline_out.to_csv("data/urban_institute/crdc_schools_states_discipline_no_sex.csv", index=False)

# Calculate disciplinary actions per enrollment without race and sex information
df_discipline_out = pd.read_csv("data/urban_institute/crdc_schools_states_discipline.csv")
df_discipline_out = df_discipline_out[df_discipline_out["year"] > 2019]
df_discipline_out = df_discipline_out.groupby(['country', 'fips', "state"], as_index=False).agg({
    "behavioral_incidents": "sum",
    "enrollment": "sum"
})
df_discipline_out["behavioral_per_enrollment"] = df_discipline_out["behavioral_incidents"] / df_discipline_out[
    "enrollment"]

# Write the DataFrame to a CSV file
df_discipline_out.to_csv("data/urban_institute/crdc_schools_states_discipline_no_sex_no_race.csv", index=False)

## Teachers information Dataset

I extract the teacher information dataset, transform it, and finally, load it into a CSV file. This dataset comprises one file encompassing all the years.

In [14]:
# Load the teacher dataset
teachers_df = pd.read_csv("https://educationdata.urban.org/csv/crdc/schools_crdc_teacher.csv")

if not teachers_df.empty:
    
    # Remove unwanted data and handle missing values       
    teachers_df.loc[teachers_df["fips"] < 0, "fips"] = np.nan
        
    teachers_df.loc[teachers_df["teachers_fte_crdc"] < 0, "teachers_fte_crdc"] = 0
    teachers_df.loc[teachers_df["teachers_certified_fte"] < 0, "teachers_certified_fte"] = 0
    teachers_df.loc[teachers_df["teachers_uncertified_fte"] < 0, "teachers_uncertified_fte"] = 0
    teachers_df.loc[teachers_df["teachers_first_year_fte"] < 0, "teachers_first_year_fte"] = 0
    teachers_df.loc[teachers_df["teachers_second_year_fte"] < 0, "teachers_second_year_fte"] = 0
    teachers_df.loc[teachers_df["teachers_absent_fte"] < 0, "teachers_absent_fte"] = 0
    
    # Add state information
    teachers_df["state"] = teachers_df['fips'].apply(lambda x: fips_dict[x] if x in fips_dict else np.nan)
    
    # Drop rows with missing state information
    teachers_df.dropna(subset=['fips', 'state'], inplace=True)
    
    # Handle missing values
    teachers_df["teachers_fte_crdc"] = teachers_df["teachers_fte_crdc"].fillna(0)
    teachers_df["teachers_certified_fte"] = teachers_df["teachers_certified_fte"].fillna(0)
    teachers_df["teachers_uncertified_fte"] = teachers_df["teachers_uncertified_fte"].fillna(0)
    teachers_df["teachers_first_year_fte"] = teachers_df["teachers_first_year_fte"].fillna(0)
    teachers_df["teachers_second_year_fte"] = teachers_df["teachers_second_year_fte"].fillna(0)
    teachers_df["teachers_absent_fte"] = teachers_df["teachers_absent_fte"].fillna(0)
    
    # Convert data types
    teachers_df["teachers_fte_crdc"] = teachers_df["teachers_fte_crdc"].astype(int)
    teachers_df["teachers_certified_fte"] = teachers_df["teachers_certified_fte"].astype(int)
    teachers_df["teachers_uncertified_fte"] = teachers_df["teachers_uncertified_fte"].astype(int)
    teachers_df["teachers_first_year_fte"] = teachers_df["teachers_first_year_fte"].astype(int)
    teachers_df["teachers_second_year_fte"] = teachers_df["teachers_second_year_fte"].astype(int)
    teachers_df["teachers_absent_fte"] = teachers_df["teachers_absent_fte"].astype(int)

    # Add country information
    teachers_df['country'] = "United States"
    # Add date information
    teachers_df["day"] = 1
    teachers_df["month"] = 1
    teachers_df['date'] = pd.to_datetime(teachers_df[["year","month","day"]])
    
    # Aggregate data at a state level
    teachers_df = teachers_df.groupby(["year", 'date', 'country', "fips", "state"], as_index=False).agg({
        "teachers_fte_crdc": "sum",
        "teachers_certified_fte": "sum",
        "teachers_uncertified_fte": "sum",
        "teachers_first_year_fte": "sum",
        "teachers_second_year_fte": "sum",
        "teachers_absent_fte": "sum",
    })


/var/folders/50/q05bl7gj7jxb0qq6rb67fp4m0000gn/T/ipykernel_9159/1036997871.py:2: DtypeWarning: Columns (0,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  teachers_df = pd.read_csv("https://educationdata.urban.org/csv/crdc/schools_crdc_teacher.csv")


In [15]:
# Read the teachers data for the year 2017 and copy as 2020
df_teachers_2020 = teachers_df[teachers_df["year"] == 2017].copy()

df_teachers_2020["year"] = 2020
df_teachers_2020["day"] = 1
df_teachers_2020["month"] = 1
df_teachers_2020['date'] = pd.to_datetime(df_teachers_2020[["year","month","day"]])
df_teachers_2020['date'] = df_teachers_2020['date'].dt.strftime("%Y-%m-%d")
df_teachers_2020.drop(columns=['day', 'month'], inplace=True)

# Save the DataFrame to a CSV file
df_teachers = pd.concat([teachers_df, df_teachers_2020])
df_teachers.to_csv("data/urban_institute/crdc_schools_states_teachers.csv", index=False)

## School Finance Dataset

Finally, I extract the School Finance information, transform it, and load it into a CSV file. This dataset comprises one file encompassing all the years.

In [16]:
# Read the finance dataset
finance_df = pd.read_csv("https://educationdata.urban.org/csv/crdc/schools_crdc_finance.csv")

if not finance_df.empty:
    
    # Handle missing or negative values
    finance_df.loc[finance_df["fips"] < 0, "fips"] = np.nan
    finance_df.loc[finance_df["salaries_teachers"] < 0, "salaries_teachers"] = 0
    finance_df.loc[finance_df["salaries_total"] < 0, "salaries_total"] = 0
    finance_df.loc[finance_df["expenditures_nonpersonnel"] < 0, "expenditures_nonpersonnel"] = 0
    finance_df.loc[finance_df["support_fte"] < 0, "support_fte"] = 0
    finance_df.loc[finance_df["administration_fte"] < 0, "administration_fte"] = 0
    finance_df.loc[finance_df["salaries_support"] < 0, "salaries_support"] = 0
    finance_df.loc[finance_df["salaries_administration"] < 0, "salaries_administration"] = 0
    
    # Assign state information
    finance_df["state"] = finance_df['fips'].apply(lambda x: fips_dict[x] if x in fips_dict else np.nan)
    
    # Drop rows with missing state information
    finance_df.dropna(subset=['fips', 'state'], inplace=True)
    
    # Convert columns to integer type
    finance_df["salaries_teachers"] = finance_df["salaries_teachers"].fillna(0)
    finance_df["salaries_total"] = finance_df["salaries_total"].fillna(0)
    finance_df["expenditures_nonpersonnel"] = finance_df["expenditures_nonpersonnel"].fillna(0)
    finance_df["support_fte"] = finance_df["support_fte"].fillna(0)
    finance_df["administration_fte"] = finance_df["administration_fte"].fillna(0)
    finance_df["salaries_support"] = finance_df["salaries_support"].fillna(0)
    finance_df["salaries_administration"] = finance_df["salaries_administration"].fillna(0)
    
    finance_df["salaries_teachers"] = finance_df["salaries_teachers"].astype(int)
    finance_df["salaries_total"] = finance_df["salaries_total"].astype(int)
    finance_df["expenditures_nonpersonnel"] = finance_df["expenditures_nonpersonnel"].astype(int)
    finance_df["support_fte"] = finance_df["support_fte"].astype(int)
    finance_df["administration_fte"] = finance_df["administration_fte"].astype(int)
    finance_df["salaries_support"] = finance_df["salaries_support"].astype(int)
    finance_df["salaries_administration"] = finance_df["salaries_administration"].astype(int)
    
    # Add country information and date
    finance_df['country'] = "United States"
    finance_df["day"] = 1
    finance_df["month"] = 1
    finance_df['date'] = pd.to_datetime(finance_df[["year","month","day"]])
    
    # Group data by year, state, and FIPS code and calculate aggregates
    finance_df = finance_df.groupby(["year", 'date', 'country', "fips", "state"], as_index=False).agg({
        "salaries_teachers": ["sum", "mean"],
        "salaries_total": ["sum", "mean"],
        "expenditures_nonpersonnel": "sum",
        "support_fte": "sum",
        "administration_fte": "sum",
        "salaries_support": ["sum", "mean"],
        "salaries_administration": ["sum", "mean"],
    })
    
    # Flatten column names
    flat_cols = []
    for i in finance_df.columns:
        if i[1]:
            flat_cols.append(i[0]+'_'+i[1])
        else:
            flat_cols.append(i[0])
    finance_df.columns = flat_cols

    # Write the DataFrame to a CSV file
    finance_df.to_csv(f"data/raw/urban_institute/crdc_schools_states_finance_all.csv", index=False)

/var/folders/50/q05bl7gj7jxb0qq6rb67fp4m0000gn/T/ipykernel_9159/2780407147.py:2: DtypeWarning: Columns (0,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  finance_df = pd.read_csv("https://educationdata.urban.org/csv/crdc/schools_crdc_finance.csv")


In [17]:
# Read the school finance data for the year 2017 and copy as 2020
df_finance_2020 = finance_df[finance_df["year"] == 2017].copy()
df_finance_2020["year"] = 2020
df_finance_2020["day"] = 1
df_finance_2020["month"] = 1
df_finance_2020['date'] = pd.to_datetime(df_finance_2020[["year","month","day"]])
df_finance_2020['date'] = df_finance_2020['date'].dt.strftime("%Y-%m-%d")
df_finance_2020.drop(columns=['day', 'month'], inplace=True)

df_finance = pd.concat([finance_df, df_finance_2020])

In [18]:
# Read the teacher and population datasets
df_teachers = pd.read_csv("./data/urban_institute/crdc_schools_states_teachers.csv")
df_population = pd.read_csv("./data/kff/population.csv")
df_population = df_population.groupby(["year", "country", "state"], as_index=False).agg({"population": "sum"})

# Merge finance and population datasets based on year, country, and state
df_finance_merged = df_finance.merge(right=df_population, on=["year", "country", "state"])

# Calculate financial metrics per GDP
df_finance_merged["salaries_teaches_per_gdp"] =  df_finance_merged["salaries_teachers_sum"] / df_finance_merged["population"]
df_finance_merged["expenditures_nonpersonnel_per_gdp"] =  df_finance_merged["expenditures_nonpersonnel_sum"] / df_finance_merged["population"]
df_finance_merged["salaries_support_per_gdp"] =  df_finance_merged["salaries_support_sum"] / df_finance_merged["population"]
df_finance_merged["salaries_administration_per_gdp"] =  df_finance_merged["salaries_administration_sum"] / df_finance_merged["population"]
df_finance_merged["others_salaries_sum"] = (df_finance_merged["salaries_total_sum"] - df_finance_merged["salaries_teachers_sum"] - df_finance_merged["salaries_support_sum"] 
                                            - df_finance_merged["salaries_administration_sum"])
df_finance_merged["others_salaries_per_gdp"] =  df_finance_merged["others_salaries_sum"] / df_finance_merged["population"]

# Write the merged finance DataFrame to a CSV file
df_finance_merged.to_csv("./data/urban_institute/crdc_schools_states_finance.csv", index=False)

In [19]:
# Select relevant columns from the merged finance DataFrame for pie chart data
df_finance_pie = df_finance_merged[df_finance_merged["year"] > 2019][["year", "country", "state", "salaries_teaches_per_gdp", "expenditures_nonpersonnel_per_gdp", "salaries_support_per_gdp", "salaries_administration_per_gdp"]]

# Rename columns for better readability
df_finance_pie = df_finance_pie.rename(columns={
    "salaries_administration_per_gdp": "Administration Salaries per Capita",
    "salaries_support_per_gdp": "Support Salaries per Capita",
    "expenditures_nonpersonnel_per_gdp": "Other Expenditures per Capita",
    "salaries_teaches_per_gdp": "Teacher Salaries per Capita"
})

# Reshape the DataFrame for pie chart visualization
df_finance_pie = df_finance_pie.melt(id_vars=["year", "country", "state"], value_vars=["Administration Salaries per Capita", "Support Salaries per Capita", "Other Expenditures per Capita", "Teacher Salaries per Capita"], var_name='category', value_name='expenditures')

# Write the reshaped DataFrame to a CSV file for pie chart visualization
df_finance_pie.to_csv("./data/urban_institute/crdc_schools_states_finance_pie.csv", index=False)

Here I obtain the data for the map showing teachers per 1000 people.

In [20]:
# Select relevant columns from the finance DataFrame
df_finance_temp = df_finance[["year", "country", "state", "salaries_teachers_sum"]]
# Filter teachers DataFrame for years greater than 2019
df_teachers = df_teachers[df_teachers["year"] > 2019]

# Merge teachers DataFrame with finance DataFrame based on year, country, and state
df_teachers = df_teachers.merge(right=df_finance_temp, on= ["year", "country", "state"] )
# Merge teachers DataFrame with population DataFrame based on year, country, and state
df_teachers = df_teachers.merge(right=df_population, on= ["year", "country", "state"] )

# Calculate total number of teachers
df_teachers["total_teachers"] = df_teachers["teachers_fte_crdc"] + df_teachers["teachers_certified_fte"] + df_teachers["teachers_first_year_fte"] + df_teachers["teachers_second_year_fte"]

# Calculate number of teachers per 1000 people
df_teachers["teachers_per_1000people"] = df_teachers["total_teachers"] * 1000 / df_teachers["population"]
df_teachers["teachers_per_1000people"] = df_teachers["teachers_per_1000people"].astype(int)

# Write the DataFrame to a CSV file
df_teachers.to_csv("./data/urban_institute/crdc_schools_states_teachers_per_1000.csv", index=False)